In [ ]:
import tensorflow as tf
import numpy      as np

import directoryFunctions
import pathlib
import config
import random
import data
import time

from tensorflow.keras.optimizers  import Adam
from tensorflow.keras.callbacks   import CSVLogger, EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.losses      import CategoricalCrossentropy
from tensorflow.keras.layers      import Dense, Dropout, LSTM
from tensorflow.keras.models      import load_model, Sequential

"""
Documentation:
- numpy
    1. array()
        - https://numpy.org/doc/stable/reference/generated/numpy.array.html?highlight=array#numpy.array
    2. ceil()
        - https://numpy.org/doc/stable/reference/generated/numpy.ceil.html?highlight=ceil#numpy.ceil
    3. load()
        - https://numpy.org/doc/stable/reference/generated/numpy.load.html?highlight=load#numpy.load
    4. pad()
        - https://numpy.org/doc/stable/reference/generated/numpy.pad.html?highlight=pad#numpy.pad
- pathlib
    1. Path(), /, glob()
        - https://docs.python.org/3/library/pathlib.html
- random
    1. shuffle()
        - https://docs.python.org/3/library/random.html#random.shuffle
- tensorflow
    - data.Dataset
        1. batch(), cache(), from_tensor_slices(), prefetch(), repeat(), shuffle()
            - https://www.tensorflow.org/versions/r2.1/api_docs/python/tf/data/Dataset
    - keras
        - callbacks
            1. CSVLogger(), EarlyStopping(), ModelCheckpoint(), TensorBoard()
                - https://www.tensorflow.org/versions/r2.1/api_docs/python/tf/keras/callbacks
        - layers
            1. Dense()
                - https://www.tensorflow.org/versions/r2.1/api_docs/python/tf/keras/layers/Dense
            2. Dropout()
                - https://www.tensorflow.org/versions/r2.1/api_docs/python/tf/keras/layers/Dropout
            3. LSTM()
                - https://www.tensorflow.org/versions/r2.1/api_docs/python/tf/keras/layers/LSTM
        - losses
            1. CategoricalCrossentropy()
                - https://www.tensorflow.org/versions/r2.1/api_docs/python/tf/keras/losses/CategoricalCrossentropy
        - models
            1. load_model()
                - https://www.tensorflow.org/versions/r2.1/api_docs/python/tf/keras/models/load_model
            2. Sequential()
                1. add(), compile(), fit()
                    - https://www.tensorflow.org/versions/r2.1/api_docs/python/tf/keras/Sequential
        - optimizers
            1. Adam()
                - https://www.tensorflow.org/versions/r2.1/api_docs/python/tf/keras/optimizers/Adam
- time
    1. time()
        - https://docs.python.org/3/library/time.html#time.time

Sources:
    1. Loading numpy arrays
        - https://www.tensorflow.org/tutorials/load_data/numpy
        - https://www.tensorflow.org/guide/data#consuming_numpy_arrays
    2. Preparing data for training
        - https://www.tensorflow.org/tutorials/load_data/images#basic_methods_for_training
        - https://www.tensorflow.org/guide/data_performance
"""

In [ ]:
"""
Function Name: getLabeledSequences
Number of parameters: 1
List of parameters:
    1. datasetType | str | Determines whether the returned data (sequences) are from the Train, Validation or Test set.
Pre-condition:
    1. 'datasetType' must be either 'Train', 'Validation', or 'Test'. Otherwise empty arrays are returned.
Post-condition:
    1. Returns a numpy array of sequences (np.float32) and a numpy array of corresponding labels (np.uint8).
"""
def getLabeledSequences(datasetType):
    maxFrameCount = dataObj.getMaxFrameCount()
    sequences = []
    labels    = []
    dataRows = dataObj.data[:]
    random.shuffle(dataRows)
    sequencesPath = pathlib.Path(config.Config().sequencesPath)
    for dataRow in dataRows:
        if(datasetType == dataRow[0]):
            sequencePath = sequencesPath/dataRow[0]/dataRow[1]/(dataRow[2] + "_featureSequence.npy")
            sequence     = np.load(sequencePath)
            sequence     = np.pad(sequence, ((0, maxFrameCount - int(dataRow[3])), (0, 0)), 'edge')
            sequences.append(sequence)
            label = dataObj.getClassIndex(dataRow[1])
            labels.append(label)
    return np.array(sequences), np.array(labels, dtype=np.uint8)

In [ ]:
"""
Function Name: getDataset
Number of parameters: 2
List of parameters:
    1. sequences | np.array | Numpy array of sequences.
    2. labels    | np.array | Numpy array of labels.
Pre-condition: n/a
Post-condition:
    1. Returns a tf.data.Dataset object based on the 'sequences' and 'labels' arrays. 
"""
def getDataset(sequences, labels):
    dataset = tf.data.Dataset.from_tensor_slices((sequences, labels))
    return dataset

In [ ]:
"""
Function Name: prepareTrainDataset
Number of parameters: 3
List of parameters:
    1. dataset           | tf.data.Dataset | Dataset object that contains the processed images and their labels.
    2. cache             | bool/str        | If False or an empty string, cache is not used. 
                                             If True then dataset is cached in memory.
                                             Otherwise, dataset is cached in a cache file.
    3. shuffleBufferSize | int             | Size of the shuffle buffer.
Pre-condition:
    1. If 'cache' is a non-empty string, then it must be a directory that exists.
Post-condition:
    1. If specified, the dataset will be cached (either on memory or on disk).
    2. Prefetches the next batched dataset.
    3. Returns a shuffled and batched dataset.
"""
def prepareTrainDataset(dataset, cache, shuffleBufferSize):
    if cache:
        if isinstance(cache, str):
            dataset = dataset.cache(cache)
        else:
            dataset = dataset.cache()
    
    dataset = dataset.shuffle(buffer_size = shuffleBufferSize)
    # Repeat forever
    dataset = dataset.repeat()
    dataset = dataset.batch(BATCH_SIZE)
    # `prefetch` lets the dataset fetch batches in the background while the model is training.
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    
    return dataset

In [ ]:
"""
Function Name: getModel
Number of parameters: 0
List of parameters: n/a
Pre-condition: n/a
Post-condition:
    1. Returns a compiled model (tf.keras.models.Sequential object).
"""
def getModel():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(4096, input_shape=(None, 2048), dropout=0.5))
    model.add(tf.keras.layers.Dropout(0.15))
    model.add(tf.keras.layers.Dense(512, activation = 'relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(numClasses, activation = 'softmax')) # change 3 to numClasses
    
    model.compile(optimizer = tf.keras.optimizers.Adam(lr = 0.0000025),
                  loss      = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
                  metrics   = ['accuracy'])
    return model

In [ ]:
"""
Function Name: trainModel
Number of parameters: 8
List of parameters:
    1. model             | tf.keras.models | Model to be trained.
    2. epochs            | int             | Epoch to stop training.
    3. trainDataset      | tf.data.Dataset | Data used for training.
    4. validationDataset | tf.data.Dataset | Data used for validation.
    5. steps_per_epoch   | int             | Total number of steps in a 'training' epoch.
    6. validation_steps  | int             | Total number of steps in a 'validation' epoch.
    7. callbacks         | list            | List of callbacks. (tf.keras.callbacks.Callback)
Pre-condition: n/a
Post-condition:
    1. Returns the model and History object.
"""
def trainModel(model, epochs, trainDataset, validationDataset, steps_per_epoch, validation_steps, callbacks):
    history = model.fit(trainDataset, 
                        epochs = epochs,
                        validation_data  = validationDataset,
                        steps_per_epoch  = steps_per_epoch,
                        validation_steps = validation_steps,
                        callbacks = callbacks)
    return model, history

In [ ]:
dataObj    = data.Data()
numClasses = dataObj.numClasses
AUTOTUNE   = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32

In [ ]:
"""
Function Name: main
Number of parameters: 0
List of parameters: n/a
Pre-condition: n/a
Post-condition:
    1. Trains a model and saves callbacks to disk. 
    2. Nothing is returned.
"""
def main():
    cf         = config.Config()
    rootPath   = pathlib.Path(cf.rootPath)
    
    rnnCallbacksDirectory    = rootPath/'Callbacks'/'RNN'/f'{numClasses}'
    modelCheckpointDirectory = rnnCallbacksDirectory/'ModelCheckpoint'
    tensorboardDirectory     = rnnCallbacksDirectory/'Tensorboard'
    csvLoggerDirectory       = rnnCallbacksDirectory/'CSVLogger'
    
    directoryFunctions.createDirectory(csvLoggerDirectory)
    directoryFunctions.createDirectory(modelCheckpointDirectory)
    
    trainSequences, trainLabels           = getLabeledSequences('Train')
    validationSequences, validationLabels = getLabeledSequences('Validation')
    
    trainDataset      = getDataset(trainSequences, trainLabels)
    validationDataset = getDataset(validationSequences, validationLabels)
    
    trainSeqCount      = dataObj.getDatasetCount('Train')
    validationSeqCount = dataObj.getDatasetCount('Validation')
    
    cachePath     = pathlib.Path(r"./Cache")
    cacheFilePath = cachePath/'trainRNNDatasetCache'
    
    directoryFunctions.removeDirectory(cachePath)
    directoryFunctions.createDirectory(cachePath)
    
    trainDataset      = prepareTrainDataset(trainDataset, str(cacheFilePath), int(trainSeqCount*(0.1)))
    validationDataset = validationDataset.batch(BATCH_SIZE)
    
    steps_per_epoch  = np.ceil(trainSeqCount/BATCH_SIZE)
    validation_steps = np.ceil(validationSeqCount/BATCH_SIZE)
    
    currTime = int(time.time())
    modelCheckpointName = f'{currTime}' + '_RNN_{epoch:03d}_{val_loss:.2f}.h5'
    modelCheckpoint = ModelCheckpoint(filepath       = str(modelCheckpointDirectory/modelCheckpointName),
                                      save_best_only = True)
    tensorboard     = TensorBoard(log_dir = str(tensorboardDirectory/f'{currTime}'))
    csvLogger       = CSVLogger(str(csvLoggerDirectory/f'{currTime}.log'))
    earlyStopping   = EarlyStopping(monitor = 'val_loss', patience = 5)
    callbacks       = [modelCheckpoint, tensorboard, csvLogger, earlyStopping]
    
    epochs = 5
    
    savedModelPath = "" # insert path to saved model (.h5 file) here
    if savedModelPath == "":
        model = getModel()
    else:
        model  = load_model(savedModelPath)
        epochs = 5
    
    trained_model, history = trainModel(model, epochs, 
                                        trainDataset, validationDataset, 
                                        steps_per_epoch, validation_steps, callbacks)
    
    directoryFunctions.removeDirectory(cachePath)

In [ ]:
main()